In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud
from konlpy.tag import Hannanum
from konlpy.tag import Okt
from konlpy.tag import Komoran
import sqlite3 as sq3
from pandas.io import sql
import os
import numpy as np
import math

## read db file

In [2]:
# 데이터베이스 저장
def db_save(data, db_name, table_name):
    with sq3.connect(os.path.join('.',db_name)) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            data.to_sql(name = table_name, con = con, index = False, if_exists='replace') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(data), '건 저장완료..')

In [3]:
# 데이터베이스 불러오기
def db_select(db_name, table_name):
    with sq3.connect(db_name) as con: 
        try:
            query = 'SELECT * FROM {}'.format(table_name)
            df = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e)) 
        return df  

### Save dictionaries

In [4]:
import pickle

In [5]:
def saveDict(my_dict, file_name):
    with open(file_name, "wb") as tf:
        pickle.dump(my_dict,tf)
    print(len(my_dict),' saved to file...')

In [6]:
def readDict(file_name):
    new_dict = dict()
    with open(file_name, "rb") as tf:
        new_dict = pickle.load(tf)
    return(new_dict)

# save word tokens into a file

In [7]:
DB_NAME = '법무법인_세종_뉴스레터.db'
TABLE_NAME = 'emerics_tokenized2'
df = db_select(DB_NAME,TABLE_NAME)
df1 = df.iloc[:,[0,4]].copy()
df1

,제목,BoW
0,Policy and Law Report_Vol.153 – 정부 주요 정책 및 입법정...,"세종,입법전략자문팀Legislative,정책,입법,동향,레터,정부,국회,접수,의안현..."
1,순환자원으로 인정받을 수 있는 폐기물의 범위가 확대됩니다 - ‘자원순환기본법 시행령...,"환경부,순환자원,인정,요건,완화,골자,자원순환기본법,시행령,시행령,개정안,입법예고,..."
2,상속세 및 증여세법상 성실공익법인의 요건으로 전용계좌의 신고의무는 포함되지 않는다는...,"서울고등법원,성실공익법인,요건,공익법인,전용계좌,개설,사용,전용계좌,지연,사안,상속..."
3,검사의 수사개시 범위 관련 개정 대통령령(2022.9.10.자)의 주요 내용,"검사,수사개시,축소,개정,형사소송법,검찰청법,시행,대통령령,검사,수사개시,범죄,개정..."
4,Policy and Law Report_Vol.152 – 정부 주요 정책 및 입법정...,"세종,입법전략자문팀Legislative,정책,입법,동향,레터,정부,국회,접수,의안현..."
...,...,...
903,상해에서 적격 외국유한파트너(QFLP)제도 시범실시,"상해시금융서비스판공실上海市金融服务办公室,상해시,정부당국,본시,외상,투자지분투자기업,..."
904,소득세법상 특수관계자간 상장주식 양도시 시가,"대법원,소득세법상,상장주식,시가,산정,판결,본건판결,사안,개요,상장법인,최대주주,개..."
905,“미네르바”법 위헌결정과 기업법무에서의 시사점,"글로벌,금융위,미네르바라,필명,인터넷,논객,화제,다음Daum,아고라,경제토론방,우리..."
906,소위 ‘물량 몰아주기’에 관한 공정거래위원회의 심사기준 개정,"공정거래위원회,부당한,지원행위,심사지침,시행,현저한,규모,거래,부당지원행위일명,물량..."


# 단어의 총 등장 횟수 구하기 --> TF

In [8]:
def dict_each(string):
    each_art = dict()     # dictionary for each article
    temp_list = []        # temporary list for the word tokens
    
    for i in string.split(','):
        temp_list.append(i)

    for each_t in temp_list:
        if each_t in each_art.keys():
            each_art[each_t] += 1
        else :
            each_art[each_t] = 1
            
    t_list = sorted(each_art.items(), key=lambda x:x[1], reverse=True)
    each_TF = dict(t_list)
    return each_TF

In [9]:
total_TF = []

for i in df1.iloc[:,1]:
    tokens_const = []
    total_TF.append(dict_each(i))

len(total_TF)

908

In [10]:
saveDict(total_TF,'TF_file.pkl')
#saveDict(total_TF,'TF-IDF_file.pkl')

908  saved to file...


In [11]:
TF_new = readDict('TF_file.pkl')
DF = readDict('DF_file.pkl')

# TF-IDF
##### 참고: https://brunch.co.kr/@bflysoft1117/127, https://www.researchgate.net/publication/342833072_gujojeog_topig_modelling-eul_hwal-yonghan_gong-gongdeiteo_suyo_bunseog_Analysis_of_Open_Government_Data_Demand_Using_Structural_Topic_Modeling
높을수록 단어 중요도가 높다
### TF-IDF = TF X log(N/DF)
- TF: 문서 내 특정 단어의 빈도 수
- N: 분석 대상 문서 통합
- DF: 특정 단어를 포함하는 문서빈도 수

In [12]:
def TF_IDF_each(dct):
    TF_IDF_i = dict()     # dictionary to save TF-IDF values
    
    for eachkey in list(dct.keys()):
        TF_temp = dct[eachkey]
        DF_temp = DF[eachkey]
        TF_IDF_1 = TF_temp*math.log(N/DF_temp)
        TF_IDF_i[eachkey] = TF_IDF_1

    temp_tf_idf = sorted(TF_IDF_i.items(), key=lambda x:x[1], reverse=True)
    TF_IDF_temp = dict(temp_tf_idf)
    return TF_IDF_temp

In [13]:
TF_IDF = []     # dictionary to save TF-IDF values

N = len(df1)
for each_article in total_TF:
    each_TI = TF_IDF_each(each_article)
    TF_IDF.append(each_TI)

# Find the 10 keywords of the first article
- ordered by importance

In [14]:
def get_each_keywords(dictionary):
    keyword_s = ""
    
    temp_list = sorted(dictionary.items(), key=lambda x:x[1], reverse=True)
    temp_2 = dict(temp_list)
    keywords = list(temp_2.keys())[:10]
    keyword_s = ','.join(keywords)
    
    return keyword_s

In [19]:
TF_IDF_final = []

for each in TF_IDF:
    TF_IDF_final.append(get_each_keywords(each))
    
df['키워드10개'] = TF_IDF_final
df_final = df.drop(['tokens','counts'], axis=1)
df_final

,제목,날짜,주제,본문,BoW,키워드10개
0,Policy and Law Report_Vol.153 – 정부 주요 정책 및 입법정...,2022.09.20,입법전략자문,법무법인유 세종 입법전략자문팀Legislative and Government Aff...,"세종,입법전략자문팀Legislative,정책,입법,동향,레터,정부,국회,접수,의안현...","법안,동향,정책,국회,입법전략자문팀Legislative,입법행정예고,입법,향후,출신,도움"
1,순환자원으로 인정받을 수 있는 폐기물의 범위가 확대됩니다 - ‘자원순환기본법 시행령...,2022.09.16,환경,환경부는 순환자원 인정 요건 완화를 골자로 하는 자원순환기본법 시행령 일부개정안이하...,"환경부,순환자원,인정,요건,완화,골자,자원순환기본법,시행령,시행령,개정안,입법예고,...","순환자원,폐기물,자원순환기본법,인정,시행령,요건,개정안,배출,인정제도,유상거래"
2,상속세 및 증여세법상 성실공익법인의 요건으로 전용계좌의 신고의무는 포함되지 않는다는...,2022.09.13,조세,최근 서울고등법원은 성실공익법인의 다른 요건을 갖춘 공익법인이 전용계좌를 개설하여 ...,"서울고등법원,성실공익법인,요건,공익법인,전용계좌,개설,사용,전용계좌,지연,사안,상속...","전용계좌,성실공익법인,상증세법,개설,엄격해석,초과보유,조세법률주의상,가산세,특수관계,원고"
3,검사의 수사개시 범위 관련 개정 대통령령(2022.9.10.자)의 주요 내용,2022.09.13,형사,2022 9 10 검사의 직접 수사개시 범위를 축소시키는 내용 등이 포함된 개정 형...,"검사,수사개시,축소,개정,형사소송법,검찰청법,시행,대통령령,검사,수사개시,범죄,개정...","범죄,수사개시,검사,경제범죄,부패범죄,수사,대통령령,검찰청법,형사소송법,개시"
4,Policy and Law Report_Vol.152 – 정부 주요 정책 및 입법정...,2022.09.13,입법전략자문,법무법인유 세종 입법전략자문팀Legislative and Government Aff...,"세종,입법전략자문팀Legislative,정책,입법,동향,레터,정부,국회,접수,의안현...","법안,동향,정책,국회,입법전략자문팀Legislative,입법행정예고,입법,향후,출신,도움"
...,...,...,...,...,...,...
903,상해에서 적격 외국유한파트너(QFLP)제도 시범실시,2011.02,중국,2011년 1월 11일 상해시금융서비스판공실上海市金融服务办公室 등 상해시 관련 정부...,"상해시금융서비스판공실上海市金融服务办公室,상해시,정부당국,본시,외상,투자지분투자기업,...","실시방법,중국,상해시,위안화,경외,사모펀드,상해,정부당국,시범실시,외자PE회사"
904,소득세법상 특수관계자간 상장주식 양도시 시가,2011.02,조세,2011년 1월 13일 대법원에서는 소득세법상 상장주식의 시가의 산정과 관련하여 중...,"대법원,소득세법상,상장주식,시가,산정,판결,본건판결,사안,개요,상장법인,최대주주,개...","시가,최종시세가액,상증세법,가액,한국거래소,법인세법,개인,양도,거래,상장주식"
905,“미네르바”법 위헌결정과 기업법무에서의 시사점,2011.02,General,2008년 7월 글로벌 금융위기가 발생하였을 때 미네르바라는 필명의 한 인터넷 논객...,"글로벌,금융위,미네르바라,필명,인터넷,논객,화제,다음Daum,아고라,경제토론방,우리...","공익,헌법재판소,미네르바,무죄판결,통신,전기통신기본법,규제법령,목적,추상적,파견근로자"
906,소위 ‘물량 몰아주기’에 관한 공정거래위원회의 심사기준 개정,2011.01,공정거래,공정거래위원회는 2010 12 22 부당한 지원행위 심사지침을 개정2011 1 1 ...,"공정거래위원회,부당한,지원행위,심사지침,시행,현저한,규모,거래,부당지원행위일명,물량...","지원행위,지원객체,현저한,규모,물량,거래,부당지원행위,공정거래위원회,지원주체,유리한"


In [20]:
db_save(df_final, DB_NAME, '키워드')

908 건 저장완료..
